# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846943371155                   -0.70    4.8         
  2   -7.852336811934       -2.27       -1.53    1.0   30.5ms
  3   -7.852615502768       -3.55       -2.56    1.2   31.8ms
  4   -7.852645972440       -4.52       -2.88    2.2   39.5ms
  5   -7.852646480547       -6.29       -3.15    1.0   29.7ms
  6   -7.852646679291       -6.70       -4.03    1.2   96.0ms
  7   -7.852646686459       -8.14       -5.06    1.8   36.3ms
  8   -7.852646686725       -9.58       -5.50    2.2   39.1ms
  9   -7.852646686728      -11.58       -5.64    1.2   31.4ms
 10   -7.852646686730      -11.65       -6.39    1.0   30.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846834451538                   -0.70           4.5         
  2   -7.852554973348       -2.24       -1.64   0.80    2.2    327ms
  3   -7.852638985353       -4.08       -2.74   0.80    1.0   28.1ms
  4   -7.852646441595       -5.13       -3.35   0.80    2.2   36.5ms
  5   -7.852646678981       -6.62       -4.20   0.80    1.8   33.6ms
  6   -7.852646686483       -8.12       -4.75   0.80    1.8   33.4ms
  7   -7.852646686718       -9.63       -5.69   0.80    1.8   32.8ms
  8   -7.852646686730      -10.95       -6.47   0.80    2.0   36.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.413061e+01     3.449337e+00
 * time: 0.07423806190490723
     1     1.348093e+00     1.764888e+00
 * time: 0.3639180660247803
     2    -1.469790e+00     1.930433e+00
 * time: 0.3942830562591553
     3    -3.909347e+00     1.768220e+00
 * time: 0.4377319812774658
     4    -5.362253e+00     1.438126e+00
 * time: 0.5726280212402344
     5    -6.955596e+00     9.841543e-01
 * time: 0.6147270202636719
     6    -7.105859e+00     1.260686e+00
 * time: 0.644287109375
     7    -7.442948e+00     1.133851e+00
 * time: 0.6745750904083252
     8    -7.710794e+00     3.718770e-01
 * time: 0.7174930572509766
     9    -7.772794e+00     3.184127e-01
 * time: 0.7474701404571533
    10    -7.813965e+00     1.086521e-01
 * time: 0.7775819301605225
    11    -7.836733e+00     1.603111e-01
 * time: 0.8076629638671875
    12    -7.845131e+00     4.106174e-02
 * time: 0.8383331298828125
    13    -7.848828e+00     2.953831e-02
 * time: 0.8687410354614

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846894393802                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645903828                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.46s
  3   -7.852646686730      -13.30       -7.24    212ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.7088708055714735e-7
|ρ_newton - ρ_scfv| = 1.594199404291166e-7
|ρ_newton - ρ_dm|   = 1.0099508606861262e-9
